In [203]:
import pandas as pd
import numpy as np
import random
import calendar

In [204]:
n_vacancies = 200
min_date = pd.to_datetime('2018/01/01')
max_date = pd.to_datetime('2018/12/31')

In [205]:
# generate all vacancy data

def generate_vacancy_data(n_vacancies):
    vacancy_df = pd.DataFrame()
    # create random ids for each row
    vacancy_df['ID'] = random.sample(range(1,n_vacancies+1), n_vacancies)
    # randomly add values for FP and BU Region columns
    vacancy_df['FP'] = np.random.choice(['FT','PT'], n_vacancies, replace=True)
    regions = ['North', 'South', 'East', 'West', 'Central', 'Midwest', 'Northwest']
    vacancy_df['BU Region'] = np.random.choice(regions, n_vacancies, replace=True)
    
    # get the number of days in the time period
    n_days = (max_date - min_date).days + 1
    # generate random approved dates
    vacancy_df['Approved'] = min_date + pd.to_timedelta(np.random.randint(n_days, size=n_vacancies), unit='d')
    # add empty column filled with NaT
    vacancy_df['On hold'] = pd.NaT
    # generate 1% on hold vacancies by selecting 1% of rows
    a = vacancy_df.sample(frac=0.01)
    # generate random on hold dates for the selected rows in 10-30 days after approved date
    a['On hold'] = a['Approved'] + pd.to_timedelta(np.random.randint(low=10, high=30, size=a.shape[0]), unit='d')
    # replace modified rows in original dataset
    vacancy_df.loc[a.index, 'On hold'] = a['On hold'].dt.date
    
    # generate sourcing start date to be within 5-10 days after approved date
    vacancy_df['Sourcing start'] = vacancy_df['Approved'] + pd.to_timedelta(np.random.randint(low=5, high=10, size=n_vacancies), unit='d')
    # generate Interview start date to be within 10-20 days after sourcing start date
    vacancy_df['Interview start'] = vacancy_df['Sourcing start'] + pd.to_timedelta(np.random.randint(low=10, high=20, size=n_vacancies), unit='d')
    # generate Interview end date to be within 15-30 days after interview start date
    vacancy_df['Interview end'] = vacancy_df['Interview start'] + pd.to_timedelta(np.random.randint(low=15, high=30, size=n_vacancies), unit='d')
    # generate Offered date to be within 5-10 days after interview end date
    vacancy_df['Offered'] = vacancy_df['Interview end'] + pd.to_timedelta(np.random.randint(low=5, high=10, size=n_vacancies), unit='d')
    # generate Filled date to be within 5-10 days after offered date
    vacancy_df['Filled'] = vacancy_df['Offered'] + pd.to_timedelta(np.random.randint(low=5, high=10, size=n_vacancies), unit='d')
    
    # remove values for on hold vacancies
    vacancy_df.loc[a.index, ['Sourcing start', 'Interview start', 'Interview end', 'Offered', 'Filled']] = pd.NaT
    
    return vacancy_df

In [206]:
# generate complete dataset
vacancy_df = generate_vacancy_data(n_vacancies)
vacancy_df

,ID,FP,BU Region,Approved,On hold,Sourcing start,Interview start,Interview end,Offered,Filled
0,40,PT,West,2018-01-13,NaT,2018-01-19,2018-02-07,2018-03-04,2018-03-09,2018-03-15
1,20,FT,Midwest,2018-08-20,NaT,2018-08-25,2018-09-06,2018-10-02,2018-10-08,2018-10-16
2,76,PT,Northwest,2018-10-19,NaT,2018-10-26,2018-11-08,2018-12-05,2018-12-14,2018-12-21
3,26,FT,Northwest,2018-12-03,NaT,2018-12-08,2018-12-18,2019-01-16,2019-01-24,2019-02-01
4,130,PT,Central,2018-10-13,NaT,2018-10-20,2018-10-31,2018-11-29,2018-12-06,2018-12-15
...,...,...,...,...,...,...,...,...,...,...
195,183,PT,Central,2018-07-12,NaT,2018-07-18,2018-07-31,2018-08-23,2018-08-31,2018-09-09
196,49,PT,North,2018-10-27,NaT,2018-11-01,2018-11-13,2018-12-09,2018-12-14,2018-12-22
197,144,FT,West,2018-07-14,NaT,2018-07-19,2018-07-30,2018-08-25,2018-09-01,2018-09-07
198,165,FT,Midwest,2018-02-06,NaT,2018-02-15,2018-03-04,2018-03-26,2018-04-01,2018-04-06


In [213]:
# create data files for each month

def create_monthly_df(month):
    # get start date for the month
    month_start = pd.to_datetime('2018/'+str(month)+'/01')
    # get end date for the month
    month_end = month_start + pd.to_timedelta(calendar.monthrange(2018, month)[1]-1, unit='d')
    
    # create monthly data
    monthly_df = vacancy_df.copy()
    
    # replace dates after month end date with NaT
    monthly_df[monthly_df.columns[3:]] = monthly_df.iloc[:,3:].where(monthly_df.iloc[:,3:]<=month_end, pd.NaT)
    
    # replace rows with NaT if Filled date is in a previous month
    monthly_df[monthly_df.columns[3:]] = monthly_df.iloc[:,3:].where(
        (pd.isnull(monthly_df.iloc[:,-1])) | (monthly_df.iloc[:,-1]>=month_start), pd.NaT)
    
    # remove rows with all blank values using index
    empty_index = monthly_df[monthly_df.iloc[:,3:].isnull().all(axis=1)].index
    monthly_df = monthly_df.drop(index=empty_index).reset_index()

    # convert On hold column to datetime to be able to compare in the next step with idxmax
    monthly_df['On hold'] = pd.to_datetime(monthly_df['On hold'], errors='coerce')
    # add status column by using the max date value from the specified columns
    monthly_df['Status'] = monthly_df[['Approved', 'On hold', 'Sourcing start', 
                                       'Interview start', 'Interview end', 'Offered', 'Filled']].idxmax(axis=1)
    # remove previous index column 
    monthly_df = monthly_df.drop(['index'], axis=1)
    
    # remove NaT values
    monthly_df = monthly_df.fillna('')
    
    # write to csv
    file_name = 'Vacancy data/'+'2018-'+str(month)+'.csv'
    monthly_df.to_csv(file_name, index=False)

In [214]:
# generate monthly files
for i in range(12):
    create_monthly_df(i+1)